# [E-13] Recommand

이전 스텝에서 배운 MF 모델 학습 방법을 토대로, 내가 좋아할 만한 영화 추천 시스템을 제작해 보겠습니다.

이번에 활용할 데이터셋은 추천 시스템의 MNIST라고 부를만한 Movielens 데이터입니다.

- 유저가 영화에 대해 평점을 매긴 데이터가 데이터 크기 별로 있습니다. MovieLens 1M Dataset 사용을 권장합니다.
- 별점 데이터는 대표적인 explicit 데이터입니다. 하지만 implicit 데이터로 간주하고 테스트해 볼 수 있습니다.
- 별점을 시청횟수로 해석해서 생각하겠습니다.
- 또한 유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정하고 제외하겠습니다.

In [1]:
import numpy as np
import scipy
import implicit
import pandas as pd

print(np.__version__)
print(scipy.__version__)
print(implicit.__version__)

1.23.5
1.9.1
0.4.8


# 1) 데이터 준비와 전처리

In [2]:
import os
rating_file_path='./Data/iu/movies/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [3]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['ratings']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [4]:
# ratings 컬럼의 이름을 counts로 바꿉니다.
ratings.rename(columns={'ratings':'counts'}, inplace=True)

In [5]:
ratings['counts']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: counts, Length: 836478, dtype: int64

In [6]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path='./Data/iu/movies/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [7]:
# 출시 년도 컬럼 추가
movies['publish_year'] = movies['title'].apply(lambda x: x.split(' (')[1].replace(')', ''))
# title
movies['title'] = movies['title'].apply(lambda x: x.split(' (')[0])
movies['title'] = movies['title'].apply(lambda x: x.replace(', The', ''))

In [8]:
movies.head()

,movie_id,title,genre,publish_year
0,1,Toy Story,Animation|Children's|Comedy,1995
1,2,Jumanji,Adventure|Children's|Fantasy,1995
2,3,Grumpier Old Men,Comedy|Romance,1995
3,4,Waiting to Exhale,Comedy|Drama,1995
4,5,Father of the Bride Part II,Comedy,1995


In [9]:
movies[movies['movie_id']==2858]

,movie_id,title,genre,publish_year
2789,2858,American Beauty,Comedy|Drama,1999


In [10]:
movies[movies['title']=='American Beauty']

,movie_id,title,genre,publish_year
2789,2858,American Beauty,Comedy|Drama,1999


모델 활용을 위한 전처리

In [11]:
# 고유한 유저, 아티스트를 찾아내는 코드
user_unique = ratings['user_id'].unique()
movies_unique = movies['movie_id'].unique()

# 유저, 아티스트 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movies_unique)}

In [12]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = ratings['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(ratings):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    ratings['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# artist_to_idx을 통해 artist 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_movie_data = ratings['movie_id'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(ratings):
    print('movie column indexing OK!!')
    ratings['movie_id'] = temp_movie_data
else:
    print('movie column indexing Fail!!')

ratings

user_id column indexing OK!!
movie column indexing OK!!


,user_id,movie_id,counts,timestamp
0,0,1176,5,978300760
1,0,655,3,978302109
2,0,902,3,978301968
3,0,3339,4,978300275
4,0,2286,5,978824291
...,...,...,...,...
1000203,6038,1074,3,956715518
1000205,6038,1078,5,956704887
1000206,6038,558,5,956704746
1000207,6038,1080,4,956715648


In [13]:
movies['movie_id'] = movies['movie_id'].map(movie_to_idx.get)

In [14]:
movies

,movie_id,title,genre,publish_year
0,0,Toy Story,Animation|Children's|Comedy,1995
1,1,Jumanji,Adventure|Children's|Fantasy,1995
2,2,Grumpier Old Men,Comedy|Romance,1995
3,3,Waiting to Exhale,Comedy|Drama,1995
4,4,Father of the Bride Part II,Comedy,1995
...,...,...,...,...
3878,3878,Meet the Parents,Comedy,2000
3879,3879,Requiem for a Dream,Drama,2000
3880,3880,Tigerland,Drama,2000
3881,3881,Two Family House,Drama,2000


In [15]:
# idx_to_movie  movie_to_idx
movies_list = movies[['movie_id', 'title']]
idx_to_title = movies_list.set_index('movie_id').T.to_dict('records')[0]
title_to_idx = {v:k for k,v in idx_to_title.items()}

# 2) 분석해 봅시다.

- ratings에 있는 유니크한 영화 개수
- ratings에 있는 유니크한 사용자 수
- 가장 인기 있는 영화 30개(인기순)

In [16]:
# ratings에 있는 유니크한 영화 개수
ratings['movie_id'].nunique()

3628

In [17]:
# ratings에 있는 유니크한 사용자 수
ratings['user_id'].nunique()

6039

In [18]:
# 가장 인기 있는 영화 30개(인기순)
favorite_movies = ratings.groupby('movie_id')['user_id'].count().reset_index(name='count')
favorite_movies_30 = favorite_movies.sort_values(by='count', ascending=False).head(30)

In [19]:
favorite_movies_30['title'] = favorite_movies_30['movie_id'].map(idx_to_title.get)

In [20]:
idx_to_title.get(2789)

'American Beauty'

In [21]:
movies[movies['movie_id']==2789]

,movie_id,title,genre,publish_year
2789,2789,American Beauty,Comedy|Drama,1999


In [22]:
favorite_movies_30

,movie_id,count,title
2600,2789,3211,American Beauty
249,257,2910,Star Wars: Episode IV - A New Hope
1080,1178,2885,Star Wars: Episode V - The Empire Strikes Back
1094,1192,2716,Star Wars: Episode VI - Return of the Jedi
1810,1959,2561,Saving Private Ryan
569,585,2509,Terminator 2: Judgment Day
573,589,2498,Silence of the Lambs
1082,1180,2473,Raiders of the Lost Ark
1152,1250,2460,Back to the Future
2325,2502,2434,Matrix


In [23]:
user_count = ratings.groupby('user_id')['movie_id'].count()
user_count.describe()

count    6039.000000
mean      138.512668
std       156.241599
min         1.000000
25%        38.000000
50%        81.000000
75%       177.000000
max      1968.000000
Name: movie_id, dtype: float64

# 3) 내가 선호하는 영화를 5가지 골라서 ratings에 추가해 줍시다.

In [24]:
# 본인이 좋아하시는 아티스트 데이터로 바꿔서 추가하셔도 됩니다! 단, 이름은 꼭 데이터셋에 있는 것과 동일하게 맞춰주세요. 
my_favorite = ['Matrix' , 'Toy Story' ,'Back to the Future' ,'Sixth Sense' ,'Forrest Gump']
user_id = 6039

# 'zimin'이라는 user_id가 위 아티스트의 노래를 30회씩 들었다고 가정하겠습니다.
my_playlist = pd.DataFrame({'user_id': [user_id]*5, 'movie_id':[title_to_idx.get(x) for x in my_favorite ], 'counts':[5]*5})

if not ratings.isin({'user_id':[user_id]})['user_id'].any():  
    ratings = ratings.append(my_playlist)                  

ratings.tail(10)       # 잘 추가되었는지 확인해 봅시다.

C:\Users\peunj\AppData\Local\Temp\ipykernel_44740\1591240798.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ratings = ratings.append(my_playlist)


,user_id,movie_id,counts,timestamp
1000203,6038,1074,3,956715518.0
1000205,6038,1078,5,956704887.0
1000206,6038,558,5,956704746.0
1000207,6038,1080,4,956715648.0
1000208,6038,1081,4,956715569.0
0,6039,2502,5,NaN
1,6039,0,5,NaN
2,6039,1250,5,NaN
3,6039,2693,5,NaN
4,6039,352,5,NaN


In [25]:
pd.DataFrame({'user_id': user_id*5, 'movie_id':[title_to_idx.get(x) for x in my_favorite ], 'counts':[5]*5})

,user_id,movie_id,counts
0,30195,2502,5
1,30195,0,5
2,30195,1250,5
3,30195,2693,5
4,30195,352,5


# 4) CSR matrix를 직접 만들어 봅시다.

In [26]:
# 실습 위에 설명보고 이해해서 만들어보기
from scipy.sparse import csr_matrix

num_user = ratings['user_id'].nunique() # 6040
num_movies = movies['movie_id'].nunique() # 3883)

csr_data = csr_matrix((ratings.counts, (ratings.user_id, ratings.movie_id)), shape= (num_user, num_movies))
csr_data

<6040x3883 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

# 5) als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 봅시다.

In [27]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [28]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.transpose()
csr_data_transpose

<3883x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [29]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [30]:
# 모델 훈련 - cpu 6s 
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

# 6) 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.

In [31]:

toy = title_to_idx['Toy Story']
zimin_vector, toy_vector = als_model.user_factors[user_id], als_model.item_factors[toy]


In [32]:
zimin_vector

array([ 0.20298727,  0.8963866 ,  0.59801376, -1.4290588 , -0.2641769 ,
        0.06771204, -0.46456558,  0.01525942, -0.39747083, -0.06443864,
       -0.5076811 ,  0.32272348,  0.12088138, -0.58135146,  0.09368981,
       -0.21573311,  0.3416047 , -0.0237206 , -0.5091346 ,  0.02464702,
       -0.35880256,  1.3778784 ,  0.59493756,  0.56245655,  0.36262497,
       -0.6491861 , -1.2739525 ,  0.0331216 ,  0.5319014 ,  0.39865807,
       -0.55510974, -0.4055066 , -0.00339062, -0.01093223,  0.06011061,
       -0.40749338,  0.01657504, -0.98186016,  0.24443093,  0.634577  ,
       -0.14733554, -0.64180577, -0.04810932,  0.33317935,  0.326639  ,
        0.70457506, -0.46641228, -0.05468026,  0.5311709 ,  0.22753152,
        0.72394013,  0.4709201 ,  0.18372187,  0.91329473, -0.13950212,
        0.22973089, -0.55594176,  1.0632452 ,  0.05321424,  0.4991332 ,
       -0.21523377,  0.38242096,  0.6973526 , -0.01421243,  0.33623022,
        0.07136278,  0.7786003 , -0.42193398,  0.43288952, -0.98

In [33]:
toy_vector

array([-0.00628074,  0.02452082,  0.01431118, -0.00655353, -0.01662683,
        0.00921569, -0.03530374, -0.00031503, -0.0048253 , -0.00557367,
       -0.03254396,  0.02620954,  0.02815942, -0.01229301,  0.0292167 ,
        0.00434934,  0.04532408, -0.01923417, -0.00488471, -0.01840832,
        0.01732271,  0.04391017,  0.03882004,  0.0167543 ,  0.00895662,
        0.0081269 , -0.0220867 , -0.01405068,  0.0122287 ,  0.00607974,
       -0.01675272,  0.03445741,  0.03464834,  0.01441531,  0.02123307,
        0.00306402,  0.01242415, -0.0245939 ,  0.01609116,  0.00929788,
       -0.00111523,  0.00975877,  0.00115206,  0.00562822,  0.00576246,
        0.02507339, -0.0071828 ,  0.0009757 ,  0.01707861, -0.0023911 ,
        0.0180979 ,  0.0148247 ,  0.040006  , -0.00866238,  0.01778787,
        0.01111937,  0.00466145,  0.01312524,  0.02178068,  0.01419767,
        0.00697621,  0.00364943,  0.00051851,  0.04426594,  0.02940319,
        0.01239761,  0.02069503, -0.01487451, -0.00968978, -0.04

In [34]:
# 내가 좋아하는 영화와 나의 벡터 내적 수치
np.dot(zimin_vector, toy_vector)

0.56758904

In [35]:
# 이외의 영화에서 한편 골라 나의 벡터 내적 수치
beauty = title_to_idx['American Beauty']
matrix_vector = als_model.item_factors[beauty]
np.dot(zimin_vector, matrix_vector)

0.23648301

- 🧐내가 좋아하는 영화 'Toy Story'는 나의 선호도(벡터내적) 0.56로 높게 나와서 선호도가 높다는 것을 알 수 있다.    
- 반면, 'American Beauty'는 내적이 0.23로 Toy Story에 비해서 선호도가 적은 수치로 나왔다. 

# 7) 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.

In [36]:
favorite_movies = 'Matrix'
movie_id = title_to_idx[favorite_movies]

similar_movies = als_model.similar_items(movie_id, N=15)
{idx_to_title[i[0]]:i[1] for i in similar_movies}

{'Matrix': 1.0,
 'Terminator 2: Judgment Day': 0.76242423,
 'Total Recall': 0.67549115,
 'Fugitive': 0.6167998,
 'Fifth Element': 0.6024013,
 'Face/Off': 0.557425,
 'Terminator': 0.55599296,
 'Jurassic Park': 0.5287898,
 'Twelve Monkeys': 0.49156907,
 'Men in Black': 0.48544908,
 'Hunt for Red October': 0.46647438,
 'Star Wars: Episode IV - A New Hope': 0.4313569,
 'Running Free': 0.41331014,
 'Sixth Sense': 0.41029626,
 'Sunchaser': 0.38181925}

🧐 Matrix 와 유사한 영화를 추천받았더니, Terminator2가 0.76의 유사도로 추천되었다. 

# 8) 내가 가장 좋아할 만한 영화들을 추천받아 봅시다.

In [37]:
#유저 선호도
user = user_id
# recommend에서는 user*item CSR Matrix를 받습니다.
movies_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)

{idx_to_title[i[0]]:i[1] for i in movies_recommended}

{'Toy Story 2': 0.49940765,
 'Groundhog Day': 0.41083717,
 'Fugitive': 0.3909694,
 'Silence of the Lambs': 0.36906385,
 'Terminator 2: Judgment Day': 0.35759136,
 'Ghostbusters': 0.33553585,
 'Saving Private Ryan': 0.3025729,
 'E.T. the Extra-Terrestrial': 0.29150346,
 'Galaxy Quest': 0.27874237,
 'Jurassic Park': 0.2764074,
 'Star Wars: Episode I - The Phantom Menace': 0.2730895,
 'Star Wars: Episode VI - Return of the Jedi': 0.27029157,
 "Schindler's List": 0.250542,
 'Shawshank Redemption': 0.2504195,
 'Shakespeare in Love': 0.24856696,
 'Star Wars: Episode IV - A New Hope': 0.24445653,
 'American Beauty': 0.23648301,
 'Terminator': 0.2343079,
 'Star Wars: Episode V - The Empire Strikes Back': 0.21912202,
 'Good Will Hunting': 0.21777822}

🧐사용자 선호도로 0.49의 Toy Story 2가 추천되었다. 

In [38]:
# 아이템 유사도
def get_similar_movie(artist_name: str):
    movie_id = title_to_idx.get(artist_name)
    similar_movie = als_model.similar_items(movie_id)
    similar_movie = {idx_to_title[i[0]]:i[1]  for i in similar_movie}
    return similar_movie


In [39]:
get_similar_movie('Groundhog Day')

{'Groundhog Day': 1.0000001,
 'Clueless': 0.60362196,
 'Forrest Gump': 0.58480585,
 'Toy Story': 0.56111926,
 'Four Weddings and a Funeral': 0.5500736,
 "There's Something About Mary": 0.54449505,
 'Shakespeare in Love': 0.5291765,
 'Pleasantville': 0.5225523,
 'Dave': 0.50999945,
 "Wayne's World": 0.49343055}

In [43]:
clueless = title_to_idx['Clueless']
explain = als_model.explain(user, csr_data, itemid=clueless)
[(idx_to_title[i[0]], i[1]) for i in explain[1]]

[('Forrest Gump', 0.10625235169091479),
 ('Toy Story', 0.03518555509417658),
 ('Back to the Future', -0.0024533862018954154),
 ('Matrix', -0.017171686998171194),
 ('Sixth Sense', -0.03884361745293282)]

In [41]:
# 기여도
Clueless = title_to_idx['Clueless']
(total_score,contributions, weights) = als_model.explain(user, csr_data, itemid=Clueless)
for i in range(len(contributions)):
    print(f'{idx_to_title.get(contributions[i][0])} 기여도:  {contributions[i][1]}')

Forrest Gump 기여도:  0.10625235169091479
Toy Story 기여도:  0.03518555509417658
Back to the Future 기여도:  -0.0024533862018954154
Matrix 기여도:  -0.017171686998171194
Sixth Sense 기여도:  -0.03884361745293282


🧐 영화 Groundhog Day 에 대한 유사 영화와 Clueless에 대한 기여도를 평가했다. 


# 회고

- CSR metrix를 처음에 만들때 차원이 맞지 않아 원인을 찾는라 힘들었지만, unique를 이용해서 movie_id를 새로 설정해 주면서 문제를 해결했다. 
- movie의 title을 전처리 해주어 년도를 분리해내서 title로 검색을 쉽도록 했다. 

https://stackoverflow.com/questions/46202519/keras-tokenizer-num-words-doesnt-seem-to-work